# Hyperparameter search
This notebook is dedicated to hyperparameter search for the different classifiers that we chose to use for leaves classification base on the images.

The goal is to find the best hyperparameters for each classifier using cross validation to compare the performances between the classifiers with the default hyperparameters and the classifiers with the best hyperparameters.

### Importing our own functions

In [4]:
from src.Data import Data

### Importing libraries
`numpy` is used to manipulate the data

In [5]:
import numpy as np

### Loading the data
The data is loaded from the `data` folder.

Samples are split into a training set and a test set with a custom ratio. Stratified sampling is used to ensure that the proportion of samples in each class is the same in both sets.

The number of samples in the least represented class is computed to choose the number of folds for cross-validation.

In [6]:
data: Data = Data(test_size=0.2, include_images=True)

least_populated_class_count = np.unique(data.y_train, return_counts=True)[1].min()
print("Least populated class count:", least_populated_class_count)
print("This is the maximum valid number of folds for cross validation.")

Least populated class count: 8
This is the maximum valid number of folds for cross validation.


### Choosing the models
Here you can choose which models you want to include in the hyperparameter search.

The parameter `n_jobs` is used to specify the number of cores to use for parallel processing. If `-1` is given, all cores are used.

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

classifiers = [
    RandomForestClassifier(n_jobs=-1),  # Time to fit < 5 hours. Results in report.
    # SVC(),  # Unable to hyper thread. Too long to fit.
    KNeighborsClassifier(n_jobs=-1),  # Time to fit < 5 hours. Results in report. 
    # GradientBoostingClassifier(),  # Unable to hyper thread. Too long to fit.
    # AdaBoostClassifier()  # Unable to hyper thread. Too long to fit.
]

### Getting the list of hyperparameters
To simplify the hyperparameter search, we use the `get_params()` method of the classifier to get the list of hyperparameters that can be tuned.


In [8]:
for classifier in classifiers:
    print("Classifier:", classifier.__class__.__name__)
    print("Parameters:")
    for key in classifier.get_params():
        print("\t", key)
    print("")

Classifier: RandomForestClassifier
Parameters:
	 bootstrap
	 ccp_alpha
	 class_weight
	 criterion
	 max_depth
	 max_features
	 max_leaf_nodes
	 max_samples
	 min_impurity_decrease
	 min_samples_leaf
	 min_samples_split
	 min_weight_fraction_leaf
	 n_estimators
	 n_jobs
	 oob_score
	 random_state
	 verbose
	 warm_start

Classifier: KNeighborsClassifier
Parameters:
	 algorithm
	 leaf_size
	 metric
	 metric_params
	 n_jobs
	 n_neighbors
	 p
	 weights


### Chosing the hyperparameters to tune
We then need to choose from the list above which hyperparameters we want to tune. We can also choose the range of values to test for each hyperparameter.

The `param_grid` variable is a dictionary where the keys are the names of the hyperparameters and the values are the list of values to test for each hyperparameter.

In [9]:
param_grids = []

# RandomForestClassifier
param_grid = {
    "n_estimators": [10, 50, 100, 200, 500],
    "criterion": ["gini", "entropy"],
    "min_samples_split": [2, 5, 10],
    "max_features": ["sqrt", "log2"]
}
if "RandomForestClassifier" in [classifier.__class__.__name__ for classifier in classifiers]:
    param_grids.append(param_grid)

# SVC
param_grid = {
    "kernel": ["linear", "poly", "rbf", "sigmoid"],
    "C": [0.1, 0.5, 2, 5, 10, 20, 50, 100, 200, 500, 1000],
    "gamma": ["scale", "auto"]
}
if "SVC" in [classifier.__class__.__name__ for classifier in classifiers]:
    param_grids.append(param_grid)

# KNeighborsClassifier
param_grid = {
    "n_neighbors": [1, 2, 5, 10],
    "weights": ["uniform", "distance"],
    "algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
    "leaf_size": [1, 2, 5, 10, 20, 30, 50],
    "p": [1, 2]
}
if "KNeighborsClassifier" in [classifier.__class__.__name__ for classifier in classifiers]:
    param_grids.append(param_grid)

# GradientBoostingClassifier
param_grid = {
    "learning_rate": [0.005, 0.01, 0.025, 0.05, 0.1, 0.5],
    "n_estimators": [100, 500], 
    "criterion": ["friedman_mse", "squared_error"],
    "max_depth": [1, 2, 3, 5, 10],
    "min_samples_split": [2, 5, 10, 15, 20],
    "max_features": ["sqrt", "log2"]
}
if "GradientBoostingClassifier" in [classifier.__class__.__name__ for classifier in classifiers]:
    param_grids.append(param_grid)

# AdaBoostClassifier
param_grid = {
    "n_estimators": [50, 200, 500],
    "learning_rate": [0.001, 0.01, 0.1, 0.5],
    "algorithm": ["SAMME", "SAMME.R"]
}
if "AdaBoostClassifier" in [classifier.__class__.__name__ for classifier in classifiers]:
    param_grids.append(param_grid)

### Fitting the models with all combinations of hyperparameters
We use the `GridSearchCV` class to fit the models with all combinations of hyperparameters and find the best hyperparameters for each model. 

This class uses cross-validation to evaluate the performance through an exhaustive search over the hyperparameter values space.

In [ ]:
from sklearn.model_selection import GridSearchCV

best_params = []
best_scores = []

for classifier, param_grid in zip(classifiers, param_grids):
    print("Classifier:", classifier.__class__.__name__)
    print("Parameters:")
    for key in param_grid:
        print(f"\t{key:12}: {param_grid[key]}")
    print("")
    
    grid_search = GridSearchCV(classifier, param_grid, cv=2, verbose=1, n_jobs=-1)
    grid_search.fit(data.x_image_train, data.y_train)
    best_params.append(grid_search.best_params_)
    best_scores.append(grid_search.best_score_)
    print("Best parameters:", best_params[-1])
    print(f"Best score: {best_scores[-1]:.3f}")
    print("\n###############################\n")

### Printing the results

In [ ]:
for classifier, best_param, best_scores in zip(classifiers, best_params, best_scores):
    print("Classifier:", classifier.__class__.__name__)
    print("Best parameters:")
    for key in best_param:
        print(f"\t{key:12}: {best_param[key]}")
    print(f"Best score: {best_scores:.3f}")
    print("\n###############################\n")